In [2]:
import pandas as pd
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import war

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Analysis

Microbiome composition can be shifted by diet. The experiments that have backed this conclusion involve randomly assigning mice into control and case groups, feeding them different diets, and then quantifying the composition of the microbiota in the gut. I want to know whether we are able to use a Bayesian Dirichlet-Multinomial model to quantify the uncertainty surrounding the measured proportions. 

In [5]:
microbiome = pd.read_csv('datasets/MicrobiomeWithMetadata.csv')
microbiome.head()

,Diet,Source,Donor,CollectionMet,Sex,OTU0,OTU1,OTU2,OTU3,OTU4,...,OTU6686,OTU6687,OTU6688,OTU6689,OTU6690,OTU6691,OTU6692,OTU6693,OTU6694,OTU6695
0,0,0,0,0,0,1.560000e-11,4.720000e-11,1.230000e-11,4.520000e-11,2.720000e-11,...,6.660000e-11,3.020000e-11,4.420000e-11,7.310000e-11,7.770000e-11,4.330000e-11,5.440000e-11,8.720000e-11,2.710000e-11,4.970000e-11
1,0,1,0,0,0,2.360000e-11,9.530000e-11,3.330000e-11,2.670000e-11,2.020000e-11,...,3.260000e-12,5.390000e-11,4.730000e-11,2.600000e-11,4.240000e-11,6.550000e-11,4.850000e-11,8.380000e-11,3.500000e-11,7.620000e-11
2,0,2,0,1,0,6.770000e-11,3.680000e-11,8.020000e-11,5.490000e-11,1.340000e-11,...,7.230000e-11,6.300000e-12,7.060000e-11,8.310000e-11,3.310000e-11,7.380000e-11,4.450000e-11,9.650000e-12,7.880000e-11,3.990000e-11
3,0,2,0,0,0,5.520000e-11,9.890000e-11,4.580000e-11,3.540000e-11,2.090000e-11,...,9.100000e-11,2.940000e-11,1.000000e-10,9.820000e-11,8.540000e-11,9.730000e-11,2.960000e-11,8.580000e-13,5.880000e-11,5.510000e-11
4,0,3,0,0,0,5.240000e-11,6.340000e-11,2.350000e-11,7.470000e-11,2.490000e-11,...,1.500000e-11,4.900000e-11,8.110000e-12,4.670000e-11,8.270000e-11,4.630000e-11,1.600000e-11,5.550000e-11,7.840000e-11,8.560000e-11


The metadata file that's associated with this CSV file has to be re-coded from a CSV file to a YAML file.

In [ ]:
from collections import defaultdict
md = defaultdict(dict)  # "md" stands for "metadata dictionary"

sex = ['Male', 'Female']
donor = ['HMouseLFPP', 'CONVR', 'Human', 'Fresh', 'Frozen', 'HMouseWestern', 'CONVD']
diet = ['LFPP', 'Western', 'CARBR', 'FATR', 'Suckling', 'Human']
source = ['Cecum1', 'Cecum2', 'Colon1', 'Colon2', 'Feces', 'SI1', 'SI13', 'SI15', 'SI2', 'SI5', 'SI9', 'Stomach', 'Cecum']
collection_met = ['Contents', 'Scraping']

for i, s in enumerate(sex):
    md['sex'][i] = s
    
for i, d in enumerate(donor):
    md['donor'][i] = d
    
for i, d in enumerate(diet):
    md['diet'][i] = d
    
for i, s in enumerate(source):
    md['source'][i] = s
    
for i, c in enumerate(collection_met):
    md['collection_met'][i] = c

In [20]:
import yaml

print(yaml.dump(md))

!!python/object/apply:collections.defaultdict
args: [!!python/name:builtins.dict '']
dictitems:
  collection_met: {0: Contents, 1: Scraping}
  diet: {0: LFPP, 1: Western, 2: CARBR, 3: FATR, 4: Suckling, 5: Human}
  donor: {0: HMouseLFPP, 1: CONVR, 2: Human, 3: Fresh, 4: Frozen, 5: HMouseWestern,
    6: CONVD}
  sex: {0: Male, 1: Female}
  source: {0: Cecum1, 1: Cecum2, 2: Colon1, 3: Colon2, 4: Feces, 5: SI1, 6: SI13,
    7: SI15, 8: SI2, 9: SI5, 10: SI9, 11: Stomach, 12: Cecum}



In [21]:
with open('datasets/MicrobiomeMetadataDictionary.yml', 'w+') as f:
    f.write(yaml.dump(md))

In [23]:
with open('datasets/MicrobiomeMetadataDictionary.yml', 'r+') as f:
    metadata = yaml.load(f)
    
metadata

defaultdict(dict,
            {'collection_met': {0: 'Contents', 1: 'Scraping'},
             'diet': {0: 'LFPP',
              1: 'Western',
              2: 'CARBR',
              3: 'FATR',
              4: 'Suckling',
              5: 'Human'},
             'donor': {0: 'HMouseLFPP',
              1: 'CONVR',
              2: 'Human',
              3: 'Fresh',
              4: 'Frozen',
              5: 'HMouseWestern',
              6: 'CONVD'},
             'sex': {0: 'Male', 1: 'Female'},
             'source': {0: 'Cecum1',
              1: 'Cecum2',
              2: 'Colon1',
              3: 'Colon2',
              4: 'Feces',
              5: 'SI1',
              6: 'SI13',
              7: 'SI15',
              8: 'SI2',
              9: 'SI5',
              10: 'SI9',
              11: 'Stomach',
              12: 'Cecum'}})

In [24]:
set(microbiome['Diet'].values)

{0, 1, 2, 3, 4, 5}

In [25]:
set(microbiome['Source'].values)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [26]:
otu_cols = [c for c in microbiome.columns if 'OTU' in c]

In [ ]:
with pm.Model() as dirichlet_model:
    mu = pm.HalfNormal('mu', sd=100**2)
    n_seq_reads = pm.Poisson('n_seq_reads', mu=mu, observed=healthy_reads.sum(axis=1))
    proportions = pm.Dirichlet('proportions', a=np.ones(3), shape=(3,))
    for i in range(healthy_reads.shape[0]):
        draws = pm.Multinomial(f'draws_{i}', n=healthy_reads[i].sum(), p=proportions, observed=healthy_reads[i,:])
    dirichlet_trace = pm.sample(draws=2000)
    pm.traceplot(dirichlet_trace)